## 유통고객 구매패턴 데이터 전처리

- (Aivle School Mini-project)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

- 주어진 Data
    * customers.csv : 고객정보
    * sales.csv : 판매정보
    * products.csv : 상품정보

In [4]:
customer = pd.read_csv('C:/Users/User/mini_project/mini_project_1/customers.csv')
sales = pd.read_csv('C:/Users/User/mini_project/mini_project_1/sales.csv')
products = pd.read_csv('C:/Users/User/mini_project/mini_project_1/products.csv')

In [5]:
#customer
customer.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2
0,c328222,2014-09-25,F,1960,강원도,원주시
1,c281448,2013-06-18,F,1974,강원도,원주시
2,c038336,2003-10-10,F,1968,강원도,춘천시
3,c084237,2007-03-09,F,1982,강원도,강릉시
4,c162600,2010-06-14,F,1978,강원도,속초시


In [6]:
#sales
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [7]:
#products
products.head()

,ProductID,ProductName,Category,SubCategory
0,p1052661,새우깡,간식,과자
1,p1054261,고구마스틱,간식,과자
2,p1097821,짱구,간식,과자
3,p1097831,감자칩,간식,과자
4,p1119071,뿌셔뿌셔,간식,과자


#### products 와 customer의 수치형 변수들은 기초통계량 확인하는것 의미없다고 판단
- 가입일자, 생년월일 등의 컬럼이기 때문이다.

In [8]:
#기초통계량
sales.describe().T

,count,mean,std,min,25%,50%,75%,max
OrderID,70012.0,101.857996,62.229614,3.0,49.0,95.0,145.0,385.0
Seq,70012.0,4.892233,4.255965,1.0,2.0,4.0,7.0,40.0
Qty,70012.0,1.238588,1.419127,-44.0,1.0,1.0,1.0,200.0
Amt,70012.0,2785.229218,2734.617603,-67650.0,1300.0,1950.0,3227.0,109100.0


#### Qty, Amt는 수량과 지불가격
- 최솟값이 음수가 나왔다.
- 후에 확인 필요

### 날짜데이터 전처리

In [24]:
print(customer.info())
print('-'*50)
print(sales.info())
print('-'*50)
print(products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2243 entries, 0 to 2242
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    2243 non-null   object
 1   RegisterDate  2243 non-null   object
 2   Gender        2243 non-null   object
 3   BirthYear     2243 non-null   int64 
 4   Addr1         2243 non-null   object
 5   Addr2         2243 non-null   object
dtypes: int64(1), object(5)
memory usage: 105.3+ KB
None
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70012 entries, 0 to 70011
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderID     70012 non-null  int64 
 1   Seq         70012 non-null  int64 
 2   OrderDate   70012 non-null  object
 3   ProductID   70012 non-null  object
 4   Qty         70012 non-null  int64 
 5   Amt         70012 non-null  int64 
 6   CustomerID  70012

In [9]:
customer['RegisterDate'] = pd.to_datetime(customer['RegisterDate'])
sales['OrderDate'] = pd.to_datetime(sales['OrderDate'])

In [26]:
print(customer.info())
print('-'*50)
print(sales.info())
print('-'*50)
print(products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2243 entries, 0 to 2242
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    2243 non-null   object        
 1   RegisterDate  2243 non-null   datetime64[ns]
 2   Gender        2243 non-null   object        
 3   BirthYear     2243 non-null   int64         
 4   Addr1         2243 non-null   object        
 5   Addr2         2243 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 105.3+ KB
None
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70012 entries, 0 to 70011
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderID     70012 non-null  int64         
 1   Seq         70012 non-null  int64         
 2   OrderDate   70012 non-null  datetime64[ns]
 3   ProductID   70

#### 1) 일별 매출액 확인

In [10]:
daily_sales = sales.groupby(by = 'OrderDate', as_index = False)[['Amt']].sum()
daily_sales

,OrderDate,Amt
0,2016-01-02,503234
1,2016-01-03,211202
2,2016-01-04,705195
3,2016-01-05,502803
4,2016-01-06,485984
...,...,...
442,2017-03-27,465265
443,2017-03-28,529778
444,2017-03-29,451900
445,2017-03-30,425932


#### 2) 월별 매출액 확인

In [11]:
#연도, 월 나눠서 새로운 변수로 추가
sales['Year'] = sales['OrderDate'].dt.year
sales['Month'] = sales['OrderDate'].dt.month
sales.tail()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
70007,53,4,2017-03-31,p1072601,1,4600,c337999,2017,3
70008,53,6,2017-03-31,p1178011,1,8800,c337999,2017,3
70009,55,6,2017-03-31,p1054261,1,2091,c088320,2017,3
70010,59,4,2017-03-31,p1175481,1,1300,c238056,2017,3
70011,59,5,2017-03-31,p1013161,1,2950,c238056,2017,3


In [12]:
monthly_sales = sales.groupby(by = 'Month', as_index = False)[['Amt']].sum()
monthly_sales

,Month,Amt
0,1,28931745
1,2,26927354
2,3,32696199
3,4,15129917
4,5,13976941
5,6,13113610
6,7,10717743
7,8,10176702
8,9,10496653
9,10,10003648


In [13]:
monthly_sales.min(), monthly_sales.max() 

(Month          1
 Amt      9767765
 dtype: int64,
 Month          12
 Amt      32696199
 dtype: int64)

#### 3) 요일별 매출액 확인

In [14]:
daily_sales['Weekday'] = daily_sales['OrderDate'].dt.day_name()

In [15]:
daily_sales.head()

,OrderDate,Amt,Weekday
0,2016-01-02,503234,Saturday
1,2016-01-03,211202,Sunday
2,2016-01-04,705195,Monday
3,2016-01-05,502803,Tuesday
4,2016-01-06,485984,Wednesday


#### 4) 일별 고객 1인당 평균 구매액 (객단가)

In [16]:
order_amt = sales.groupby(by = ['OrderDate', 'CustomerID'], as_index = False)[['Amt']].sum() #일별 고객별 구매액 총합
order_amt

,OrderDate,CustomerID,Amt
0,2016-01-02,c029643,2650
1,2016-01-02,c047907,1650
2,2016-01-02,c068640,3300
3,2016-01-02,c082866,12350
4,2016-01-02,c083862,9504
...,...,...,...
32649,2017-03-31,c385994,10700
32650,2017-03-31,c394131,4150
32651,2017-03-31,c397789,4250
32652,2017-03-31,c399578,10700


In [17]:
order_amt2 = order_amt.groupby(by = 'OrderDate', as_index = False)[['Amt']].mean() #일별 구매액의 평균
order_amt2

,OrderDate,Amt
0,2016-01-02,7624.757576
1,2016-01-03,5415.435897
2,2016-01-04,6716.142857
3,2016-01-05,6364.594937
4,2016-01-06,6844.845070
...,...,...
442,2017-03-27,5410.058140
443,2017-03-28,6160.209302
444,2017-03-29,5868.831169
445,2017-03-30,5679.093333


#### 5) 일별 구매 고객 수

In [47]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1


In [18]:
daily_visit = sales.groupby(by = 'OrderDate', as_index = False)[['CustomerID']].count()
daily_visit.min(), daily_visit.max()

(OrderDate     2016-01-02 00:00:00
 CustomerID                     59
 dtype: object,
 OrderDate     2017-03-31 00:00:00
 CustomerID                    317
 dtype: object)

In [19]:
order_amt2['daily_visit'] = daily_visit['CustomerID']
order_amt2

,OrderDate,Amt,daily_visit
0,2016-01-02,7624.757576,169
1,2016-01-03,5415.435897,75
2,2016-01-04,6716.142857,203
3,2016-01-05,6364.594937,155
4,2016-01-06,6844.845070,143
...,...,...,...
442,2017-03-27,5410.058140,178
443,2017-03-28,6160.209302,202
444,2017-03-29,5868.831169,142
445,2017-03-30,5679.093333,147


#### 6) 매출 상위 top10 상품

In [55]:
products.head()

,ProductID,ProductName,Category,SubCategory
0,p1052661,새우깡,간식,과자
1,p1054261,고구마스틱,간식,과자
2,p1097821,짱구,간식,과자
3,p1097831,감자칩,간식,과자
4,p1119071,뿌셔뿌셔,간식,과자


In [56]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1


In [20]:
top_amt = pd.merge(sales, products, on = 'ProductID', how = 'inner')
top_amt

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month,ProductName,Category,SubCategory
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,순두부,반찬류,두부
1,137,4,2016-01-02,p1036481,2,2100,c280590,2016,1,순두부,반찬류,두부
2,63,16,2016-01-03,p1036481,1,1050,c037915,2016,1,순두부,반찬류,두부
3,135,3,2016-01-04,p1036481,3,3150,c100815,2016,1,순두부,반찬류,두부
4,63,13,2016-01-06,p1036481,10,10500,c048405,2016,1,순두부,반찬류,두부
...,...,...,...,...,...,...,...,...,...,...,...,...
70007,89,1,2017-03-31,p1299491,1,2150,c397789,2017,3,당근,채소,뿌리채소
70008,69,5,2017-03-31,p1299491,1,2150,c268330,2017,3,당근,채소,뿌리채소
70009,121,4,2017-03-31,p1299491,1,2150,c222461,2017,3,당근,채소,뿌리채소
70010,159,9,2017-03-31,p1299491,1,2150,c275305,2017,3,당근,채소,뿌리채소


In [21]:
top_amt2 = top_amt.groupby(by = 'ProductName', as_index = False)[['Amt']].sum()
top_amt2.sort_values(by = 'Amt', ascending = False, inplace = True)
top_amt2[:11]

,ProductName,Amt
37,우유1000,18129067
23,사과_소,14078818
9,딸기_대,11011541
7,두부_대,10119037
10,딸기_소,7517664
8,두부_소,7319963
49,콩나물,6971347
39,유기농우유,6437323
50,토마토,6291001
43,참외,5491030


### 카테고리 별 매출 비중

In [23]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1


In [24]:
products.head()

,ProductID,ProductName,Category,SubCategory
0,p1052661,새우깡,간식,과자
1,p1054261,고구마스틱,간식,과자
2,p1097821,짱구,간식,과자
3,p1097831,감자칩,간식,과자
4,p1119071,뿌셔뿌셔,간식,과자


In [25]:
cate_amt = pd.merge(sales, products, on = 'ProductID', how = 'inner')
cate_amt.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month,ProductName,Category,SubCategory
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,순두부,반찬류,두부
1,137,4,2016-01-02,p1036481,2,2100,c280590,2016,1,순두부,반찬류,두부
2,63,16,2016-01-03,p1036481,1,1050,c037915,2016,1,순두부,반찬류,두부
3,135,3,2016-01-04,p1036481,3,3150,c100815,2016,1,순두부,반찬류,두부
4,63,13,2016-01-06,p1036481,10,10500,c048405,2016,1,순두부,반찬류,두부


In [28]:
cate_amt2 = cate_amt.groupby(by = 'Category', as_index = False)[['Amt']].sum().sort_values(by = 'Amt', ascending = False)
cate_amt2

,Category,Amt
4,채소,54822783
1,과일,49789339
3,유제품,45261956
2,반찬류,32204820
0,간식,12920570


In [29]:
total_sum = cate_amt2['Amt'].sum()
total_sum

194999468

In [30]:
cate_amt2['Rate'] = cate_amt2['Amt'] / total_sum  #카테고리 별 매출비율
cate_amt2

,Category,Amt,Rate
4,채소,54822783,0.281143
1,과일,49789339,0.255331
3,유제품,45261956,0.232113
2,반찬류,32204820,0.165153
0,간식,12920570,0.066260


### 요일 별 매출 비중

In [32]:
day_amt = pd.merge(sales, products, on = 'ProductID', how = 'inner')
day_amt

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month,ProductName,Category,SubCategory
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,순두부,반찬류,두부
1,137,4,2016-01-02,p1036481,2,2100,c280590,2016,1,순두부,반찬류,두부
2,63,16,2016-01-03,p1036481,1,1050,c037915,2016,1,순두부,반찬류,두부
3,135,3,2016-01-04,p1036481,3,3150,c100815,2016,1,순두부,반찬류,두부
4,63,13,2016-01-06,p1036481,10,10500,c048405,2016,1,순두부,반찬류,두부
...,...,...,...,...,...,...,...,...,...,...,...,...
70007,89,1,2017-03-31,p1299491,1,2150,c397789,2017,3,당근,채소,뿌리채소
70008,69,5,2017-03-31,p1299491,1,2150,c268330,2017,3,당근,채소,뿌리채소
70009,121,4,2017-03-31,p1299491,1,2150,c222461,2017,3,당근,채소,뿌리채소
70010,159,9,2017-03-31,p1299491,1,2150,c275305,2017,3,당근,채소,뿌리채소


In [33]:
#요일을 나타내는 열 추가
day_amt['Weekday'] = day_amt['OrderDate'].dt.day_name()
day_amt.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month,ProductName,Category,SubCategory,Weekday
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,순두부,반찬류,두부,Saturday
1,137,4,2016-01-02,p1036481,2,2100,c280590,2016,1,순두부,반찬류,두부,Saturday
2,63,16,2016-01-03,p1036481,1,1050,c037915,2016,1,순두부,반찬류,두부,Sunday
3,135,3,2016-01-04,p1036481,3,3150,c100815,2016,1,순두부,반찬류,두부,Monday
4,63,13,2016-01-06,p1036481,10,10500,c048405,2016,1,순두부,반찬류,두부,Wednesday


In [34]:
day_amt2 = day_amt.groupby(by = 'Weekday', as_index = False)['Amt'].sum()

In [35]:
day_amt2

,Weekday,Amt
0,Friday,30838811
1,Monday,34484281
2,Saturday,28683620
3,Sunday,18358536
4,Thursday,26608397
5,Tuesday,28097823
6,Wednesday,27928000


In [36]:
## 요일별 매출비중
total_week_sum = day_amt2['Amt'].sum()
total_week_sum

194999468

In [37]:
day_amt2['Rate'] = day_amt2['Amt'] / total_week_sum
day_amt2

,Weekday,Amt,Rate
0,Friday,30838811,0.158148
1,Monday,34484281,0.176843
2,Saturday,28683620,0.147096
3,Sunday,18358536,0.094147
4,Thursday,26608397,0.136454
5,Tuesday,28097823,0.144092
6,Wednesday,27928000,0.143221


### 고객 나이대 분석
 - 고객의 태어난 연도를 활용하여 고객나이(Age) 컬럼 추가
 - 위 데이터는 2016년 기준 데이터임
 - 2016 - 고객 출생연도 = Age

In [39]:
customer.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2
0,c328222,2014-09-25,F,1960,강원도,원주시
1,c281448,2013-06-18,F,1974,강원도,원주시
2,c038336,2003-10-10,F,1968,강원도,춘천시
3,c084237,2007-03-09,F,1982,강원도,강릉시
4,c162600,2010-06-14,F,1978,강원도,속초시


In [40]:
customer['Age'] = 2016 - customer['BirthYear']
customer.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age
0,c328222,2014-09-25,F,1960,강원도,원주시,56
1,c281448,2013-06-18,F,1974,강원도,원주시,42
2,c038336,2003-10-10,F,1968,강원도,춘천시,48
3,c084237,2007-03-09,F,1982,강원도,강릉시,34
4,c162600,2010-06-14,F,1978,강원도,속초시,38


In [41]:
# 연령대 구하기
customer['AgeGroup'] = customer['Age'] // 10 * 10
customer

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50
1,c281448,2013-06-18,F,1974,강원도,원주시,42,40
2,c038336,2003-10-10,F,1968,강원도,춘천시,48,40
3,c084237,2007-03-09,F,1982,강원도,강릉시,34,30
4,c162600,2010-06-14,F,1978,강원도,속초시,38,30
...,...,...,...,...,...,...,...,...
2238,c302101,2013-11-23,F,1981,충청북도,청원군,35,30
2239,c213699,2011-11-04,F,1967,충청북도,청주시,49,40
2240,c314668,2014-04-25,F,1977,충청북도,청주시,39,30
2241,c160301,2010-05-15,F,1953,충청북도,청주시,63,60


In [42]:
## Gender컬럼 기반 가변수화된 데이터프레임 생성(범주 해당 1, 아니면 0)
## Year컬럼 기반 가변수화

merged_data = pd.merge(customer, sales, on = 'CustomerID', how = 'inner')
merged_data.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,155,1,2016-02-24,p1001771,1,3300,2016,2
1,c328222,2014-09-25,F,1960,강원도,원주시,56,50,143,5,2016-03-03,p1191881,1,1750,2016,3
2,c328222,2014-09-25,F,1960,강원도,원주시,56,50,143,8,2016-03-03,p1005621,1,2900,2016,3
3,c328222,2014-09-25,F,1960,강원도,원주시,56,50,143,10,2016-03-03,p1133371,2,2000,2016,3
4,c328222,2014-09-25,F,1960,강원도,원주시,56,50,143,11,2016-03-03,p1005891,1,1950,2016,3


In [43]:
dumm_cols = ['Gender', 'Year']
encoded_date = pd.get_dummies(merged_data, columns = dumm_cols, dtype = int)
encoded_date

,CustomerID,RegisterDate,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Month,Gender_F,Gender_M,Year_2016,Year_2017
0,c328222,2014-09-25,1960,강원도,원주시,56,50,155,1,2016-02-24,p1001771,1,3300,2,1,0,1,0
1,c328222,2014-09-25,1960,강원도,원주시,56,50,143,5,2016-03-03,p1191881,1,1750,3,1,0,1,0
2,c328222,2014-09-25,1960,강원도,원주시,56,50,143,8,2016-03-03,p1005621,1,2900,3,1,0,1,0
3,c328222,2014-09-25,1960,강원도,원주시,56,50,143,10,2016-03-03,p1133371,2,2000,3,1,0,1,0
4,c328222,2014-09-25,1960,강원도,원주시,56,50,143,11,2016-03-03,p1005891,1,1950,3,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70007,c181797,2011-01-26,1981,경기도,화성시,35,30,73,1,2016-06-23,p1207951,1,2800,6,1,0,1,0
70008,c181797,2011-01-26,1981,경기도,화성시,35,30,73,2,2016-06-23,p1012751,1,1250,6,1,0,1,0
70009,c181797,2011-01-26,1981,경기도,화성시,35,30,73,5,2016-06-23,p1133371,1,800,6,1,0,1,0
70010,c181797,2011-01-26,1981,경기도,화성시,35,30,73,7,2016-06-23,p1119071,1,1091,6,1,0,1,0


## 3.고객 이탈 정의

### (1) Label 만들기
 - 고객 이탈여부에 해당하는 Target변수 생성
 - 대상고객
     * 2014~2016년 신규가입고객
     * 2016년 하반기에 한번 이상 방문 고객 대상
 - Labeling
     * 위 대상 고객 중, 2017년 1월 ~ 3월(3개월)동안 방문(구매)하지 않은 사람은 이탈로 간주
     * 이탈고객 1, 이탈하지 않은 고객 0

In [44]:
customer.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50
1,c281448,2013-06-18,F,1974,강원도,원주시,42,40
2,c038336,2003-10-10,F,1968,강원도,춘천시,48,40
3,c084237,2007-03-09,F,1982,강원도,강릉시,34,30
4,c162600,2010-06-14,F,1978,강원도,속초시,38,30


In [45]:
## 2014~2016년 신규가입고객 데이터프레임 만들기

In [47]:
cust01 = customer.loc[customer['RegisterDate'].between('2014-01-01','2016-12-31')]
cust01['RegisterDate'].unique()

<DatetimeArray>
['2014-09-25 00:00:00', '2015-07-16 00:00:00', '2016-10-15 00:00:00',
 '2016-08-18 00:00:00', '2015-12-29 00:00:00', '2014-11-21 00:00:00',
 '2015-04-12 00:00:00', '2015-05-05 00:00:00', '2015-06-17 00:00:00',
 '2015-05-12 00:00:00',
 ...
 '2015-08-07 00:00:00', '2015-12-19 00:00:00', '2016-10-27 00:00:00',
 '2014-11-01 00:00:00', '2015-06-16 00:00:00', '2014-01-24 00:00:00',
 '2014-09-29 00:00:00', '2014-05-21 00:00:00', '2015-12-18 00:00:00',
 '2014-05-17 00:00:00']
Length: 488, dtype: datetime64[ns]

In [48]:
## sales 데이터에서 2016년 하반기에 방문한 고객 데이터프레임 만들기
## 하반기는 7월 1일 ~ 12월 31일

cust02 = sales.loc[sales['OrderDate'].between('2016-07-01', '2016-12-31')]
cust02.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
29519,89,1,2016-07-01,p1030071,1,2450,c127482,2016,7
29520,89,2,2016-07-01,p1144371,1,1250,c127482,2016,7
29521,89,4,2016-07-01,p1159481,1,1650,c127482,2016,7
29522,101,1,2016-07-01,p1011291,1,273,c217179,2016,7
29523,103,1,2016-07-01,p1175481,1,1300,c222561,2016,7


### 한번이상만 방문하면 되므로 중복된 고객 제거 

In [49]:
cust02 = cust02.drop_duplicates(subset = 'CustomerID')
cust02

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
29519,89,1,2016-07-01,p1030071,1,2450,c127482,2016,7
29522,101,1,2016-07-01,p1011291,1,273,c217179,2016,7
29523,103,1,2016-07-01,p1175481,1,1300,c222561,2016,7
29526,107,9,2016-07-01,p1011291,2,546,c333911,2016,7
29527,111,3,2016-07-01,p1030071,1,2450,c373441,2016,7
...,...,...,...,...,...,...,...,...,...
55591,37,5,2016-12-29,p1175481,1,1150,c153641,2016,12
55627,97,2,2016-12-30,p1005771,2,3700,c323093,2016,12
55670,25,2,2016-12-30,p1005891,1,1950,c313636,2016,12
55836,67,5,2016-12-31,p1207281,1,3300,c188728,2016,12


#### 2014 ~ 2016년 신규가입 고객이면서 2016년 하반기에 한번이상 방문한 고객 데이터 

In [50]:
cust_churn0 = pd.merge(cust01, cust02, on = 'CustomerID', how = 'inner')
cust_churn0

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,c341674,2015-02-28,M,1973,인천광역시,중구,43,40,47,7,2016-07-01,p1234751,2,1636,2016,7
543,c316736,2014-05-17,F,1976,인천광역시,중구,40,40,193,3,2016-07-22,p1013251,1,1650,2016,7
544,c310508,2014-03-12,F,1981,전라남도,해남군,35,30,155,1,2016-07-08,p1231011,2,1818,2016,7
545,c352708,2015-06-26,F,1980,충청남도,당진시,36,30,125,3,2016-07-04,p1005771,1,1850,2016,7


#### 2) Label 만들기

In [51]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1


In [52]:
#2017년 1월 ~ 3월 구매고객 데이터
cust03 = sales.loc[sales['OrderDate'].between('2017-01-01', '2017-03-31')]
cust03

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
55976,5,2,2017-01-02,p1133371,7,7000,c271068,2017,1
55977,5,6,2017-01-02,p1012751,1,1850,c271068,2017,1
55978,5,10,2017-01-02,p1144661,1,1950,c271068,2017,1
55979,5,11,2017-01-02,p1207281,1,3300,c271068,2017,1
55980,5,12,2017-01-02,p1299491,1,1950,c271068,2017,1
...,...,...,...,...,...,...,...,...,...
70007,53,4,2017-03-31,p1072601,1,4600,c337999,2017,3
70008,53,6,2017-03-31,p1178011,1,8800,c337999,2017,3
70009,55,6,2017-03-31,p1054261,1,2091,c088320,2017,3
70010,59,4,2017-03-31,p1175481,1,1300,c238056,2017,3


In [54]:
cust03 = cust03.drop_duplicates(subset = 'CustomerID')
cust03.shape

(1388, 9)

- cust03은 2017년 3개월 잘라서 가져온 데이터
- cust_churn0는 customers' 데이터에서 2014 ~ 2016년 신규 가입 고객 cust01 + sales' 데이터에서 2016년 하반기에 방문한 고객 고객이름 기준으로 합쳐서 가져온 데이터 그중 cust02에서중복되는 고객이름 제거 -> 고객은 물건 여러개살 수 있음

- cust03에서 cust_churn0과 같은이름의 열은 제거 -> cust_churn0이 포함

In [55]:
cust03['churn'] = 0

In [56]:
cust03['churn']

55976    0
55982    0
55983    0
55984    0
55985    0
        ..
69871    0
69883    0
69888    0
69904    0
69957    0
Name: churn, Length: 1388, dtype: int64

In [57]:
cust03.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month,churn
55976,5,2,2017-01-02,p1133371,7,7000,c271068,2017,1,0
55982,9,15,2017-01-02,p1207281,1,3300,c146132,2017,1,0
55983,17,8,2017-01-02,p1012811,1,3250,c226242,2017,1,0
55984,21,1,2017-01-02,p1013161,2,5722,c140658,2017,1,0
55985,25,6,2017-01-02,p1255281,1,1182,c328839,2017,1,0


In [59]:
cust_churn0.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8


In [62]:
## 두 데이터프레임 중복되는 이름 가진 컬럼 제거 
drop_cols = ['OrderID', 'Seq', 'OrderDate', 'ProductID', 'Qty', 'Amt', 'Year', 'Month']  #2017년 이탈인지 아닌지만 보면되므로
cust03 = cust03.drop(columns = drop_cols, axis = 1)
cust03

,CustomerID,churn
55976,c271068,0
55982,c146132,0
55983,c226242,0
55984,c140658,0
55985,c328839,0
...,...,...
69871,c400253,0
69883,c044830,0
69888,c346882,0
69904,c095005,0


In [63]:
tmp = pd.merge(cust_churn0, cust03, on = 'CustomerID', how = 'left')
tmp

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,churn
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,0.0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,NaN
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,NaN
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,NaN
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,c341674,2015-02-28,M,1973,인천광역시,중구,43,40,47,7,2016-07-01,p1234751,2,1636,2016,7,0.0
543,c316736,2014-05-17,F,1976,인천광역시,중구,40,40,193,3,2016-07-22,p1013251,1,1650,2016,7,0.0
544,c310508,2014-03-12,F,1981,전라남도,해남군,35,30,155,1,2016-07-08,p1231011,2,1818,2016,7,0.0
545,c352708,2015-06-26,F,1980,충청남도,당진시,36,30,125,3,2016-07-04,p1005771,1,1850,2016,7,0.0


In [64]:
tmp.isna().sum()  ### left로 한이유 -> 17년 이탈고객들은 결측치 처리 하기 위해서

CustomerID        0
RegisterDate      0
Gender            0
BirthYear         0
Addr1             0
Addr2             0
Age               0
AgeGroup          0
OrderID           0
Seq               0
OrderDate         0
ProductID         0
Qty               0
Amt               0
Year              0
Month             0
churn           184
dtype: int64

In [65]:
## 이탈고객(결측치) 1로 채우기
tmp['churn'].fillna(1, inplace = True)

In [66]:
tmp.isna().sum()

CustomerID      0
RegisterDate    0
Gender          0
BirthYear       0
Addr1           0
Addr2           0
Age             0
AgeGroup        0
OrderID         0
Seq             0
OrderDate       0
ProductID       0
Qty             0
Amt             0
Year            0
Month           0
churn           0
dtype: int64

In [67]:
tmp.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,churn
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,0.0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1.0
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,1.0
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1.0
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,0.0


In [68]:
tmp['churn'] = tmp['churn'].astype(int)

In [69]:
tmp.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,churn
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,1
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,0


In [70]:
tmp['churn'].value_counts() # 184명의 고객 이탈

churn
0    363
1    184
Name: count, dtype: int64

#### (2) feature 추가

   * 1. 가입연수
   * 2. 최근 3개월(2016년 10월~12월) 간 방문횟수
   * 3. 최근 3개월(2016년 10월~12월) 간 구매금액

In [71]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    547 non-null    object        
 1   RegisterDate  547 non-null    datetime64[ns]
 2   Gender        547 non-null    object        
 3   BirthYear     547 non-null    int64         
 4   Addr1         547 non-null    object        
 5   Addr2         547 non-null    object        
 6   Age           547 non-null    int64         
 7   AgeGroup      547 non-null    int64         
 8   OrderID       547 non-null    int64         
 9   Seq           547 non-null    int64         
 10  OrderDate     547 non-null    datetime64[ns]
 11  ProductID     547 non-null    object        
 12  Qty           547 non-null    int64         
 13  Amt           547 non-null    int64         
 14  Year          547 non-null    int32         
 15  Month         547 non-null    int32     

In [73]:
# 가입연수 : 2016년 - RegisterDate
tmp['RegDuration'] = 2016 - tmp['RegisterDate'].dt.year

In [77]:
tmp.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,churn,RegDuration
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,0,2
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1,1
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,1,0
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1,0
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,0,1


In [79]:
## 3개월간 주문 한 데이터 만들기
temp = sales.loc[sales['OrderDate'].between('2016-10-01', '2016-12-31')]
temp.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
43455,57,6,2016-10-01,p1012951,1,1850,c155875,2016,10
43456,59,3,2016-10-01,p1054261,1,1864,c177682,2016,10
43457,63,3,2016-10-01,p1207281,1,3450,c314136,2016,10
43458,63,7,2016-10-01,p1072601,1,4600,c314136,2016,10
43459,63,8,2016-10-01,p1255281,1,1182,c314136,2016,10


In [80]:
temp = temp.drop_duplicates(subset = ['OrderID', 'OrderDate', 'CustomerID'])  # 중복된 행 제거 (1번이상 주문 방문)
temp.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
43455,57,6,2016-10-01,p1012951,1,1850,c155875,2016,10
43456,59,3,2016-10-01,p1054261,1,1864,c177682,2016,10
43457,63,3,2016-10-01,p1207281,1,3450,c314136,2016,10
43461,65,4,2016-10-01,p1089531,1,2000,c319923,2016,10
43463,69,1,2016-10-01,p1006281,1,864,c302963,2016,10


In [81]:
temp['OrderDate'].value_counts()

OrderDate
2016-10-17    108
2016-12-19    107
2016-12-06     99
2016-10-10     97
2016-11-07     93
             ... 
2016-10-16     43
2016-11-27     43
2016-12-25     41
2016-10-23     39
2016-10-30     35
Name: count, Length: 92, dtype: int64

In [83]:
## 3개월간 방문횟수 구해서 데이터프레임으로 저장
temp2 = temp.groupby(by = 'CustomerID', as_index = False)['OrderDate'].count()
temp2.rename(columns = {'OrderDate':'Visit_3M_Cnt'}, inplace = True) #컬럼 명 변경
temp2.head()

,CustomerID,Visit_3M_Cnt
0,c017487,9
1,c017503,2
2,c017517,12
3,c017522,7
4,c017526,1


In [84]:
cust_churn0.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8


In [85]:
cust_churn1 = pd.merge(cust_churn0, temp2, on = 'CustomerID', how = 'left') #붙지않는 데이터 결측치 처리되어야 하므로 left join
cust_churn1.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,Visit_3M_Cnt
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,15.0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1.0
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,3.0
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1.0
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,NaN


- Visit_3M_Cnt 결측치이면 3개월간 방문하지않은 고객들이다.

In [86]:
##3개월간 구매금액
three_month_amt = sales.loc[sales['OrderDate'].between('2016-10-01', '2016-12-31')]
three_month_amt

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
43455,57,6,2016-10-01,p1012951,1,1850,c155875,2016,10
43456,59,3,2016-10-01,p1054261,1,1864,c177682,2016,10
43457,63,3,2016-10-01,p1207281,1,3450,c314136,2016,10
43458,63,7,2016-10-01,p1072601,1,4600,c314136,2016,10
43459,63,8,2016-10-01,p1255281,1,1182,c314136,2016,10
...,...,...,...,...,...,...,...,...,...
55971,97,5,2016-12-31,p1005891,2,3900,c259362,2016,12
55972,99,1,2016-12-31,p1012751,1,1850,c350918,2016,12
55973,99,2,2016-12-31,p1159481,1,2200,c350918,2016,12
55974,99,3,2016-12-31,p1207281,1,3300,c350918,2016,12


In [89]:
three_month_amt = three_month_amt.groupby(by = 'CustomerID', as_index = False)['Amt'].sum()
three_month_amt.rename(columns = {'Amt':'Amt_3M_sum'}, inplace = True)
three_month_amt

,CustomerID,Amt_3M_sum
0,c017487,44206
1,c017503,24400
2,c017517,57971
3,c017522,22027
4,c017526,9750
...,...,...
1377,c395401,9114
1378,c395638,24802
1379,c395673,4200
1380,c396022,7955


In [87]:
cust_churn1.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,Visit_3M_Cnt
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,15.0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1.0
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,3.0
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1.0
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,NaN


In [90]:
cust_churn2 = pd.merge(cust_churn1, three_month_amt, on = 'CustomerID', how = 'left')#붙지않는 데이터 결측치 처리되어야 하므로 left join
cust_churn2.head()

,CustomerID,RegisterDate,Gender,BirthYear,Addr1,Addr2,Age,AgeGroup,OrderID,Seq,OrderDate,ProductID,Qty,Amt,Year,Month,Visit_3M_Cnt,Amt_3M_sum
0,c328222,2014-09-25,F,1960,강원도,원주시,56,50,99,1,2016-07-12,p1149401,1,909,2016,7,15.0,47979.0
1,c354310,2015-07-16,F,1964,강원도,원주시,52,50,181,3,2016-11-29,p1178011,1,8800,2016,11,1.0,8800.0
2,c390828,2016-10-15,F,1984,강원도,원주시,32,30,49,14,2016-10-15,p1149411,1,1000,2016,10,3.0,18172.0
3,c386399,2016-08-18,F,1972,강원도,원주시,44,40,121,4,2016-10-12,p1012581,3,3150,2016,10,1.0,3150.0
4,c367116,2015-12-29,F,1970,강원도,원주시,46,40,169,3,2016-08-18,p1012581,1,1050,2016,8,NaN,NaN


- 3개월 간 구매금액 없는 고객 결측치로 나온다.

In [91]:
cust_churn2.isna().sum()  ## 120명의 고객은 3개월간 방문 하지 않았고 구매하지않았다 >이탈고객

CustomerID        0
RegisterDate      0
Gender            0
BirthYear         0
Addr1             0
Addr2             0
Age               0
AgeGroup          0
OrderID           0
Seq               0
OrderDate         0
ProductID         0
Qty               0
Amt               0
Year              0
Month             0
Visit_3M_Cnt    120
Amt_3M_sum      120
dtype: int64